The following code is obtained as part of the dataset and modified to obtain additional results
(Olive et al. (2020), DOI: https://doi.org/10.5281/zenodo.3928564)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from pathlib import Path
import pandas as pd

Read all csv files from google drive


In [8]:
flightlist = pd.concat(
    pd.read_csv(file, parse_dates=["callsign", "typecode", "origin", "destination", "firstseen", "lastseen", "day"])
    for file in Path('/content/drive/MyDrive/flight data').glob("flightlist_*.csv")
)
flightlist.head()
#flightlist = flightlist.drop(columns=["Unnamed: 0"])

,callsign,number,aircraft_uid,typecode,origin,destination,firstseen,lastseen,day,latitude_1,longitude_1,altitude_1,latitude_2,longitude_2,altitude_2
0,SVA870,NaN,82eff96d-11aa-41f0-a47c-974e3158c862,B77W,NaN,RPLL,2020-02-29 00:49:11+00:00,2020-03-01 07:39:58+00:00,2020-03-01 00:00:00+00:00,22.501490,46.814016,9448.8,14.493667,120.992576,99.06
1,QFA64,QF64,92ac5351-d69f-4836-979e-68b14bccf16d,B744,YSSY,YSSY,2020-02-29 01:00:11+00:00,2020-03-01 04:39:30+00:00,2020-03-01 00:00:00+00:00,-33.971775,151.181774,0.0,-33.995889,151.188755,236.22
2,LAN602,LA602,6058c095-d439-445e-9bca-643113af5b89,B789,KMIA,KLAX,2020-02-29 01:12:29+00:00,2020-03-01 14:28:58+00:00,2020-03-01 00:00:00+00:00,25.774531,-80.404064,914.4,33.947195,-118.431358,22.86
3,MAU440,MK440,e052860c-9157-4e2c-9154-e042bd77cf18,A332,VAJJ,YPPH,2020-02-29 01:34:11+00:00,2020-03-01 01:01:52+00:00,2020-03-01 00:00:00+00:00,19.088419,72.842052,0.0,-31.941330,115.964758,0.00
4,KQA784,NaN,f4f16959-b038-4726-84f4-447f8c94e5e4,B738,VABB,FACT,2020-02-29 01:44:10+00:00,2020-03-01 19:28:15+00:00,2020-03-01 00:00:00+00:00,19.088745,72.868979,0.0,-33.966468,18.602142,38.10


Basic exploratory data analysis

In [9]:
flightlist.head()

,callsign,number,aircraft_uid,typecode,origin,destination,firstseen,lastseen,day,latitude_1,longitude_1,altitude_1,latitude_2,longitude_2,altitude_2
0,SVA870,NaN,82eff96d-11aa-41f0-a47c-974e3158c862,B77W,NaN,RPLL,2020-02-29 00:49:11+00:00,2020-03-01 07:39:58+00:00,2020-03-01 00:00:00+00:00,22.501490,46.814016,9448.8,14.493667,120.992576,99.06
1,QFA64,QF64,92ac5351-d69f-4836-979e-68b14bccf16d,B744,YSSY,YSSY,2020-02-29 01:00:11+00:00,2020-03-01 04:39:30+00:00,2020-03-01 00:00:00+00:00,-33.971775,151.181774,0.0,-33.995889,151.188755,236.22
2,LAN602,LA602,6058c095-d439-445e-9bca-643113af5b89,B789,KMIA,KLAX,2020-02-29 01:12:29+00:00,2020-03-01 14:28:58+00:00,2020-03-01 00:00:00+00:00,25.774531,-80.404064,914.4,33.947195,-118.431358,22.86
3,MAU440,MK440,e052860c-9157-4e2c-9154-e042bd77cf18,A332,VAJJ,YPPH,2020-02-29 01:34:11+00:00,2020-03-01 01:01:52+00:00,2020-03-01 00:00:00+00:00,19.088419,72.842052,0.0,-31.941330,115.964758,0.00
4,KQA784,NaN,f4f16959-b038-4726-84f4-447f8c94e5e4,B738,VABB,FACT,2020-02-29 01:44:10+00:00,2020-03-01 19:28:15+00:00,2020-03-01 00:00:00+00:00,19.088745,72.868979,0.0,-33.966468,18.602142,38.10


In [10]:
flightlist.describe()

,latitude_1,longitude_1,altitude_1,latitude_2,longitude_2,altitude_2
count,5.643897e+06,5.643897e+06,5.643897e+06,5.643743e+06,5.643743e+06,5.499513e+06
mean,3.235433e+01,-3.195415e+01,1.806254e+03,3.238804e+01,-3.210289e+01,1.787318e+03
std,2.036012e+01,8.405116e+01,3.112800e+03,2.038152e+01,8.392826e+01,3.065164e+03
min,-4.864197e+01,-1.788520e+02,-6.096000e+02,-4.883075e+01,-1.789381e+02,-9.601200e+02
25%,2.944932e+01,-9.534419e+01,0.000000e+00,2.942374e+01,-9.539528e+01,1.600200e+02
50%,3.637985e+01,-7.657453e+01,3.048000e+02,3.666789e+01,-7.647858e+01,4.191000e+02
75%,4.236122e+01,1.845580e+01,1.828800e+03,4.237795e+01,1.800139e+01,1.638300e+03
max,7.068173e+01,1.796345e+02,3.840480e+04,7.055923e+01,1.793107e+02,3.864864e+04


In [11]:
print(len(flightlist))
print(flightlist.origin.isnull().sum(axis = 0) / len(flightlist))
print(flightlist.destination.isnull().sum(axis = 0) / len(flightlist))

5643897
0.24468837755189365
0.1774585893399543


Delete all records that have no origin or destination data (this action is subject to discussion)

In [12]:
flightlist = flightlist[flightlist['origin'].notna() & flightlist['destination'].notna()]

**Make matching of countries by ICAO codes table**

Read ICAO data

In [13]:
icao_codes = pd.read_excel(Path("/content/drive/MyDrive/flight data/icao_codes.xlsx"), usecols=["ICAO", "Country"], sheet_name = "data")

In [14]:
icao_codes.head()

,ICAO,Country
0,AGAF,Solomon Islands
1,AGAR,Solomon Islands
2,AGAT,Solomon Islands
3,AGBT,Solomon Islands
4,AGGA,Solomon Islands


Join by origin

In [15]:
flightlist = pd.merge(flightlist, icao_codes, how="left", left_on = ["origin"], right_on=["ICAO"])
flightlist = flightlist.rename(columns={"Country": "country_origin"}).drop(columns=["ICAO"])

Join by destination

In [16]:
flightlist = pd.merge(flightlist, icao_codes, how="left", left_on = ["destination"], right_on=["ICAO"])
flightlist = flightlist.rename(columns={"Country": "country_destination"}).drop(columns=["ICAO"])


Explore results of join

In [17]:
flightlist.head()

,callsign,number,aircraft_uid,typecode,origin,destination,firstseen,lastseen,day,latitude_1,longitude_1,altitude_1,latitude_2,longitude_2,altitude_2,country_origin,country_destination
0,QFA64,QF64,92ac5351-d69f-4836-979e-68b14bccf16d,B744,YSSY,YSSY,2020-02-29 01:00:11+00:00,2020-03-01 04:39:30+00:00,2020-03-01 00:00:00+00:00,-33.971775,151.181774,0.0,-33.995889,151.188755,236.22,Australia,Australia
1,LAN602,LA602,6058c095-d439-445e-9bca-643113af5b89,B789,KMIA,KLAX,2020-02-29 01:12:29+00:00,2020-03-01 14:28:58+00:00,2020-03-01 00:00:00+00:00,25.774531,-80.404064,914.4,33.947195,-118.431358,22.86,United States of America,United States of America
2,MAU440,MK440,e052860c-9157-4e2c-9154-e042bd77cf18,A332,VAJJ,YPPH,2020-02-29 01:34:11+00:00,2020-03-01 01:01:52+00:00,2020-03-01 00:00:00+00:00,19.088419,72.842052,0.0,-31.941330,115.964758,0.00,NaN,Australia
3,KQA784,NaN,f4f16959-b038-4726-84f4-447f8c94e5e4,B738,VABB,FACT,2020-02-29 01:44:10+00:00,2020-03-01 19:28:15+00:00,2020-03-01 00:00:00+00:00,19.088745,72.868979,0.0,-33.966468,18.602142,38.10,India,South Africa
4,NaN,NaN,31d7ce7d-5062-42e4-89a6-00709fd2e132,B788,WBSB,WBSB,2020-02-29 03:22:11+00:00,2020-03-01 00:08:34+00:00,2020-03-01 00:00:00+00:00,4.949108,114.931293,0.0,4.941788,114.926908,22.86,Brunei,Brunei


In [18]:
print(len(flightlist))
print(flightlist.country_origin.isnull().sum(axis = 0) / len(flightlist))
print(flightlist.country_destination.isnull().sum(axis = 0) / len(flightlist))

3500136
0.1387468944063888
0.20218071526363546


**Make matching of countries by ICAO nationality prefixes**

Create prefixes for origin and destination

In [19]:
flightlist['origin_pref'] = flightlist['origin'].astype(str).str[:2]
flightlist['destination_pref'] = flightlist['destination'].astype(str).str[:2]
flightlist.head()

,callsign,number,aircraft_uid,typecode,origin,destination,firstseen,lastseen,day,latitude_1,longitude_1,altitude_1,latitude_2,longitude_2,altitude_2,country_origin,country_destination,origin_pref,destination_pref
0,QFA64,QF64,92ac5351-d69f-4836-979e-68b14bccf16d,B744,YSSY,YSSY,2020-02-29 01:00:11+00:00,2020-03-01 04:39:30+00:00,2020-03-01 00:00:00+00:00,-33.971775,151.181774,0.0,-33.995889,151.188755,236.22,Australia,Australia,YS,YS
1,LAN602,LA602,6058c095-d439-445e-9bca-643113af5b89,B789,KMIA,KLAX,2020-02-29 01:12:29+00:00,2020-03-01 14:28:58+00:00,2020-03-01 00:00:00+00:00,25.774531,-80.404064,914.4,33.947195,-118.431358,22.86,United States of America,United States of America,KM,KL
2,MAU440,MK440,e052860c-9157-4e2c-9154-e042bd77cf18,A332,VAJJ,YPPH,2020-02-29 01:34:11+00:00,2020-03-01 01:01:52+00:00,2020-03-01 00:00:00+00:00,19.088419,72.842052,0.0,-31.941330,115.964758,0.00,NaN,Australia,VA,YP
3,KQA784,NaN,f4f16959-b038-4726-84f4-447f8c94e5e4,B738,VABB,FACT,2020-02-29 01:44:10+00:00,2020-03-01 19:28:15+00:00,2020-03-01 00:00:00+00:00,19.088745,72.868979,0.0,-33.966468,18.602142,38.10,India,South Africa,VA,FA
4,NaN,NaN,31d7ce7d-5062-42e4-89a6-00709fd2e132,B788,WBSB,WBSB,2020-02-29 03:22:11+00:00,2020-03-01 00:08:34+00:00,2020-03-01 00:00:00+00:00,4.949108,114.931293,0.0,4.941788,114.926908,22.86,Brunei,Brunei,WB,WB


Read ICAO nationality prefixes data

In [20]:
country_prefix = pd.read_excel(Path("/content/drive/MyDrive/flight data/icao_nationality_prefixes.xlsx"), usecols=["prefix", "Country"], sheet_name = "data")

In [21]:
country_prefix.head()

,prefix,Country
0,AG,Solomon Islands
1,AN,Nauru
2,AY,Papua New Guinea
3,BG,Greenland
4,BI,Iceland


Join by origin and destination (1st and 2nd cells respectively)

In [22]:
flightlist = pd.merge(flightlist, country_prefix, how="left", left_on = ["origin_pref"], right_on=["prefix"])
flightlist = flightlist.rename(columns={"Country": "country_origin_2"}).drop(columns=["prefix"])

In [23]:
flightlist = pd.merge(flightlist, country_prefix, how="left", left_on = ["destination_pref"], right_on=["prefix"])
flightlist = flightlist.rename(columns={"Country": "country_destination_2"}).drop(columns=["prefix"])

In [24]:
flightlist.head()

,callsign,number,aircraft_uid,typecode,origin,destination,firstseen,lastseen,day,latitude_1,longitude_1,altitude_1,latitude_2,longitude_2,altitude_2,country_origin,country_destination,origin_pref,destination_pref,country_origin_2,country_destination_2
0,QFA64,QF64,92ac5351-d69f-4836-979e-68b14bccf16d,B744,YSSY,YSSY,2020-02-29 01:00:11+00:00,2020-03-01 04:39:30+00:00,2020-03-01 00:00:00+00:00,-33.971775,151.181774,0.0,-33.995889,151.188755,236.22,Australia,Australia,YS,YS,NaN,NaN
1,LAN602,LA602,6058c095-d439-445e-9bca-643113af5b89,B789,KMIA,KLAX,2020-02-29 01:12:29+00:00,2020-03-01 14:28:58+00:00,2020-03-01 00:00:00+00:00,25.774531,-80.404064,914.4,33.947195,-118.431358,22.86,United States of America,United States of America,KM,KL,NaN,NaN
2,MAU440,MK440,e052860c-9157-4e2c-9154-e042bd77cf18,A332,VAJJ,YPPH,2020-02-29 01:34:11+00:00,2020-03-01 01:01:52+00:00,2020-03-01 00:00:00+00:00,19.088419,72.842052,0.0,-31.941330,115.964758,0.00,NaN,Australia,VA,YP,India,NaN
3,KQA784,NaN,f4f16959-b038-4726-84f4-447f8c94e5e4,B738,VABB,FACT,2020-02-29 01:44:10+00:00,2020-03-01 19:28:15+00:00,2020-03-01 00:00:00+00:00,19.088745,72.868979,0.0,-33.966468,18.602142,38.10,India,South Africa,VA,FA,India,South Africa
4,NaN,NaN,31d7ce7d-5062-42e4-89a6-00709fd2e132,B788,WBSB,WBSB,2020-02-29 03:22:11+00:00,2020-03-01 00:08:34+00:00,2020-03-01 00:00:00+00:00,4.949108,114.931293,0.0,4.941788,114.926908,22.86,Brunei,Brunei,WB,WB,Brunei,Brunei


Fill NAs from the 1st join with values from the 2nd join

In [25]:
flightlist['country_origin'] = flightlist['country_origin'].fillna(flightlist['country_origin_2'])
flightlist['country_destination'] = flightlist['country_destination'].fillna(flightlist['country_destination_2'])

In [26]:
print(len(flightlist))
print(flightlist.country_origin.isnull().sum(axis = 0) / len(flightlist))
print(flightlist.country_destination.isnull().sum(axis = 0) / len(flightlist))

3500136
0.0994498499486877
0.14432924892061336


Remove records which didn't match with any country

In [27]:
flightlist = flightlist[flightlist['country_origin'].notna() & flightlist['country_destination'].notna()]

Delete unnecessary columns

In [28]:
flightlist = flightlist.drop(columns=["country_origin_2", "country_destination_2", "origin_pref", "destination_pref"])

Add a column that indicates whether the flight is international or not.

In [29]:
flightlist['international'] = (flightlist["country_origin"] != flightlist["country_destination"])

Remove timezone information from "day" column

In [30]:
flightlist['day'] = flightlist['day'].dt.date

In [32]:
flightlist.to_csv(Path("/content/drive/MyDrive/flight data/flightsPro.csv"), index=False)

Calculate the number of domestic flights for every country by dates

In [33]:
domestic_flights=flightlist[~flightlist.international].groupby(['country_origin', 'day'], as_index = False)['callsign'
        ].count().rename(columns = {'callsign': 'domestic_flights', 'country_origin':'country'})

Calculate the number of international departures and arrivals for every country by dates

In [34]:
international_departures=flightlist[flightlist.international].groupby(['country_origin', 'day'], as_index = False)['callsign'
        ].count().rename(columns = {'callsign': 'international_departures', 'country_origin':'country'})

In [35]:
international_arrivals=flightlist[flightlist.international].groupby(['country_destination', 'day'], as_index = False)['callsign'
        ].count().rename(columns = {'callsign': 'international_arrivals', 'country_destination':'country'})

Join all these tables

In [36]:
flights = pd.merge(domestic_flights, international_departures, how="outer", on=["country", "day"])

In [37]:
flights = pd.merge(flights, international_arrivals, how="outer", on=["country", "day"])

In [38]:
flights['total_flights'] = flights['domestic_flights'] + flights['international_departures'] + flights['international_arrivals']

In [39]:
flights.head()

,country,day,domestic_flights,international_departures,international_arrivals,total_flights
0,Afghanistan,2020-02-16,0.0,NaN,3.0,NaN
1,Afghanistan,2020-03-21,0.0,2.0,5.0,7.0
2,Afghanistan,2020-03-24,0.0,5.0,1.0,6.0
3,Afghanistan,2020-04-03,0.0,1.0,7.0,8.0
4,Afghanistan,2020-04-22,0.0,0.0,1.0,1.0


Write results to CSV and Excel

In [40]:
flights.to_csv(Path("/content/drive/MyDrive/flight data/flightsNew.csv"), index=False)